<a href="https://colab.research.google.com/github/ashuttii/ashuttii-code/blob/master/DeepLearning/MouseAndCheese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
S = ['A','B','C','D','E','F','G']
A = ['A','B','C','D','E','F','G']

R Matrix

In [ ]:
rows = cols = len(S)
action = ['up', 'down', 'right', 'left']

-100 -> Traps
-1 -> Pathway - to minimize the loss
100 -> Cheese is the goal
nan -> Walls (no action)

In [ ]:
R = np.ones((rows,cols)) * -100
available_actions = [(0,1), (1,0), (1,1), (1,2), (1,3), (1,4), (1,5), (2,1), (2,4), (2,5), (3,1), (3,3), (3,4), (3,5), (4,1), (4,3), (4,5), (5,1), (5,2), (5,3), (5,5) ]
walls = [(2,2), (2,3), (3,2), (4,2), (4,4), (5,4), (6,4)]
cheese = (4,6)

for i in available_actions:
    R[i] = -1
for i in walls:
    R[i] = np.nan
R[4,6] = 100

R

array([[-100.,   -1., -100., -100., -100., -100., -100.],
       [  -1.,   -1.,   -1.,   -1.,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   nan,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   -1.,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   -1.,   nan,   -1.,  100.],
       [-100.,   -1.,   -1.,   -1.,   nan,   -1., -100.],
       [-100., -100., -100., -100.,   nan, -100., -100.]])

In [ ]:
Q = np.zeros((rows,cols,4))

#print('Q matrix: \n\n{}'.format(Q))

Training the AI Agent using Q-Learning

In [ ]:
epsilon = 0.6 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #gamma     #discount factor for future rewards
learning_rate = 0.9 #alpha.      #the rate at which the AI agent should learn

In [ ]:
#Random Start location
def get_random_location():
    current_row = np.random.randint(rows)
    current_col = np.random.randint(cols)
    action_index = np.random.randint(0,4)
    while R[current_row, current_col] != -1.:
        current_row = np.random.randint(rows)
        current_col = np.random.randint(cols)
        action_index = np.random.randint(0,4)
    print('Starting state is {},{} and action is {}'.format(str(current_row),str(current_col),action[action_index]))
    return current_row, current_col, action_index 


In [ ]:
def immediate_rewards(c_row,c_col, action_index):
    
    reward = R[c_row,c_col]
    q_old = Q[c_row,c_col,action_index]
    q_updated = q_old + (learning_rate * (reward + discount_factor * (np.max(Q[c_row, c_col]) )- q_old))
    print('Immediate reward is {}'.format(q_updated))
    return(q_updated)
    
#     reward = rewards[row_index, column_index]
#     old_q_value = q_values[old_row_index, old_column_index, action_index]
#     temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value



In [ ]:
def get_available_actions(c_row,c_col): # 0 6
    current_available_actions = []

    if((c_row-1 >=0) and (~np.isnan(R[c_row-1, c_col] ) )):
#         print("has up")
        current_available_actions.append(0)
        
    if((c_row+1 < rows) and (~np.isnan(R[c_row+1, c_col]) ) ):
#         print("has down")
        current_available_actions.append(1)
        
        
    if((c_col+1 < cols) and (~np.isnan(R[c_row, c_col+1]))):
#         print("has right")
        current_available_actions.append(2)
        
    if((c_col-1 >=0) and (~np.isnan(R[c_row, c_col-1] ) )):
#         print("has left")
        current_available_actions.append(3)
    
    actionlist = [action[x] for x in current_available_actions]
    
    print('Available Actions are : {}'.format(actionlist))
    return(current_available_actions)
    

In [ ]:
get_available_actions(2, 5)

Available Actions are : ['up', 'down', 'right', 'left']


[0, 1, 2, 3]

In [ ]:
print(R)

[[-100.   -1. -100. -100. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100.   -1.   nan   nan   -1.   -1. -100.]
 [-100.   -1.   nan   -1.   -1.   -1. -100.]
 [-100.   -1.   nan   -1.   nan   -1.  100.]
 [-100.   -1.   -1.   -1.   nan   -1. -100.]
 [-100. -100. -100. -100.   nan -100. -100.]]


In [ ]:
def next_location(c_row, c_col, action_index):
    new_row = c_row
    new_col = c_col
    if action[action_index] == 'up' :
#         and c_row > 0
        new_row -= 1
    elif action[action_index] == 'down':
#          and c_row < rows - 1
        new_row += 1
        
    elif action[action_index] == 'right':
#          and c_col < cols - 1
        new_col += 1
    
    elif action[action_index] == 'left':
#          and c_col > 0
        new_col -= 1
    
    print('New location : {} {}'.format(new_row, new_col))
    return new_row, new_col
        
    

In [ ]:
def get_next_action(available_actions, c_row, c_col,epsilon):
  if np.random.random() < epsilon:
    temp = {}
    for i in available_actions:
        temp[i] = Q[c_row, c_col, i]
    next_action = max(temp, key=temp.get)
    #         next_action= np.argmax(Q[c_row, c_col])
  else: #choose a random action
      next_action= np.random.choice(available_actions)
  print('Choosen action {}'.format(action[next_action]))
  return next_action

In [ ]:
for i in range(200):
    temp = []
    c_row, c_col,action_index = get_random_location()
    Q[c_row, c_col,action_index] = immediate_rewards(c_row, c_col,action_index)

    while R[c_row, c_col] != R[cheese]:
        #print('Current row col {} {}'.format(str(c_row),str(c_col)))
        
        available_actions = get_available_actions(c_row, c_col)

        #next action
        next_action = get_next_action(available_actions, c_row, c_col, epsilon)
        

        old_row = c_row
        old_col = c_col

        c_row , c_col = next_location(c_row, c_col, next_action)

        reward = R[c_row,c_col]
        q_old = Q[old_row,old_col,next_action]
        q_updated = q_old + (learning_rate * (reward + discount_factor * (np.max(Q[c_row, c_col]) )- q_old))
        print('Immediate reward is {}'.format(q_updated))
        Q[old_row, old_col, next_action] = q_updated
    print('Cheeeesseeeeeeeeee')
    print(Q)


Streaming output truncated to the last 5000 lines.
  [ 79.100891    79.1000089  100.          80.019     ]
  [  0.           0.           0.           0.        ]]

 [[-99.9        -99.999       37.3464044    0.        ]
  [ 32.97638941  36.37138497  42.61260697 -65.45670585]
  [ 38.26058827  33.11806343  47.41582706  37.35479364]
  [ 54.35820247  37.97374209  43.61260111  47.83135068]
  [  0.           0.           0.           0.        ]
  [ 89.         -30.81959613  -9.36632438  79.0209    ]
  [ 99.           0.           0.           0.        ]]

 [[-99.99         0.         -99.9          0.        ]
  [ 37.30639705   0.         -65.81913692 -99.        ]
  [ 42.52739089   0.         -56.81419297 -69.77181839]
  [ 48.45843474   0.           0.         -62.36801902]
  [  0.           0.           0.           0.        ]
  [ 79.01982996   0.         -90.           0.        ]
  [  0.           0.           0.         -90.        ]]]
Starting state is 4,1 and action is down
Immedi

In [ ]:
# def get_shortest_path(start_row, start_column):
#   #return immediately if this is an invalid starting location
#   if R[start_row, start_column] == R[cheese]:
#     return []
#   else: #if this is a 'legal' starting location
#     current_row, current_column = start_row, start_column
#     shortest_path = []
#     shortest_path.append([current_row, current_column])
#   #continue moving along the path until we reach the goal (i.e., the item packaging location)
#     while R[start_row, start_column] != R[cheese]:
#       print('Not Cheese, Current location {} {}'.format(start_row, start_column))
#       #get the best action to take
#       action_index = get_next_action(current_row, current_column, 1.)

#       #move to the next location on the path, and add the new location to the list
#       current_row, current_column = next_location(current_row, current_column, action_index)
#       shortest_path.append([current_row, current_column])
#     return shortest_path

In [ ]:
def get_shortest_path(start_row, start_column):
  shortest_path = []
  shortest_path.append([start_row, start_column])
  if R[start_row, start_column] == R[cheese]:
    return []
  else: 
    current_row, current_column = start_row, start_column
    while R[current_row, current_column] != R[cheese]:
      print(current_row, current_column)
      available_actions = get_available_actions(current_row, current_column)
    
      next_action = get_next_action(available_actions, current_row, current_column, 1.)
      current_row, current_column = next_location(current_row, current_column, next_action)
      shortest_path.append([current_row, current_column])
      
      
    return shortest_path

In [ ]:
get_shortest_path(1,5)


1 5
Available Actions are : ['up', 'down', 'right', 'left']
Choosen action down
New location : 2 5
2 5
Available Actions are : ['up', 'down', 'right', 'left']
Choosen action down
New location : 3 5
3 5
Available Actions are : ['up', 'down', 'right', 'left']
Choosen action down
New location : 4 5
4 5
Available Actions are : ['up', 'down', 'right']
Choosen action right
New location : 4 6


[[1, 5], [2, 5], [3, 5], [4, 5], [4, 6]]

In [ ]:
available_action = get_available_actions(0,1)
print(available_action)

next_action = get_next_action(available_action , 0,1, 1.)
print(next_action)

current_row, current_column = next_location(0,1, next_action)
print(current_row, current_column)

In [ ]:
Q[1,5]

array([33.70952112, 47.90019804, 35.0513696 , 54.94805362])

In [ ]:
Q[1,5]

array([-43.95072002,  70.19      , -43.09016067,  54.95389891])

In [ ]:
Q[4,6]

array([0., 0., 0., 0.])

In [ ]:
available_actions = get_available_actions(0,5)

Available Actions are : ['down', 'right', 'left']


In [ ]:
Q

array([[[   0.        ,   -0.999     ,   -0.99      ,    0.        ],
        [   0.        ,   30.38830545,  -90.        ,  -99.        ],
        [   0.        ,   -0.9999    ,  -99.        ,   -0.999     ],
        [   0.        ,   47.92829399,  -90.        ,  -99.        ],
        [   0.        ,   -0.9       ,  -90.        ,  -90.        ],
        [   0.        ,   -1.719     ,  -90.        ,  -90.        ],
        [   0.        ,  -90.        ,    0.        ,  -90.        ]],

       [[ -91.08009   ,  -99.999     ,   -1.9       ,    0.        ],
        [  20.76890226,   -1.        ,   41.81433207,   -1.        ],
        [ -99.99108   ,    0.        ,   48.3061374 ,   37.81573128],
        [ -51.49056262,    0.        ,   54.94894706,   -1.        ],
        [ -90.        ,   62.17069261,   -1.719     ,   49.35844393],
        [ -99.        ,   38.72532688,  -99.        ,   53.85280035],
        [ -90.        ,  -90.        ,    0.        ,   -2.38239   ]],

       [[  -0.99

In [ ]:
temp = {}
for i in available_actions:
    temp[i] = Q[c_row, c_col, i]
max_key = max(temp, key=temp.get)
max_key, temp

(1, {1: 0.0, 2: 0.0, 3: 0.0})

In [ ]:
get_available_actions(0,6)

Available Actions are : ['down', 'right']


[1, 2]

In [ ]:
c,r,a = get_random_location()
immediate_rewards(c,r,a)

Starting state is 5,2 and action is right
Immediate reward is -0.9


-0.9

In [ ]:
R[c_row, c_col] != R[cheese]

True

In [ ]:
R[c_row, c_col]

-100.0

In [ ]:
available_actions = get_available_actions(c_row, c_col)
available_actions

current row col 0 1
[1, 2, 3]


In [ ]:
np.random.choice(list(available_actions))

TypeError: 'NoneType' object is not iterable

In [ ]:
type(available_actions)

NoneType

In [ ]:
R

array([[-100.,   -1., -100., -100., -100., -100., -100.],
       [  -1.,   -1.,   -1.,   -1.,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   nan,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   -1.,   -1.,   -1., -100.],
       [-100.,   -1.,   nan,   -1.,   nan,   -1.,  100.],
       [-100.,   -1.,   -1.,   -1.,   nan,   -1., -100.],
       [-100., -100., -100., -100.,   nan, -100., -100.]])

In [ ]:
R[4,2] is np.nan

False

In [ ]:
R[cheese]

100.0

In [ ]:
    reward = R[c_row,c_col]
    q_old = Q[c_row,c_col,action_index]
    q_updated = q_old + (learning_rate * (reward + discount_factor * (np.max(Q[c_row, c_col]) )- q_old))
    print('Immediate reward is {}'.format(q_updated))

Immediate reward is nan


In [ ]:
R[c_row, c_col]

-1.0

In [ ]:
reward

-1.0

In [ ]:
q_old

nan

In [ ]:
action_index

1

In [ ]:
Q[c_row,c_col]

array([nan, nan, nan, nan])

In [ ]:
Q

array([[[ -90.        ,           nan,           nan,           nan],
        [  -9.59504447,           nan,           nan,           nan],
        [ -90.        ,           nan,  -90.        ,           nan],
        [-171.9       , -196.73133269, -187.59471544, -107.97528148],
        [-246.429     , -109.77153636, -306.79749   , -196.06584322],
        [-314.25039   , -221.13416092, -375.9678549 , -120.16464544],
        [-375.9678549 , -384.39858151, -383.57449209, -295.44948041]],

       [[          nan,           nan,           nan,           nan],
        [          nan,           nan,           nan,           nan],
        [-101.7906926 ,           nan,           nan,   -9.05137212],
        [-188.88403708,           nan,           nan,   -9.0236307 ],
        [ -90.09      ,           nan,           nan,   -8.88351545],
        [-221.35402593,           nan,  -90.        ,   -8.734731  ],
        [ -90.        , -100.719     ,  -90.        ,   -1.92279432]],

       [[       